## 04 텍스트 분류 실습 - 20 뉴스그룹 분류

In [1]:
# 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 된다.
# 이러한 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱 회기, 선형 서포트 벡터 머신, 나이브 베이즈 등이다.
# 텍스트를 기반으로 분류를 수행할 때는 먼저 텍스트를 정규화한 뒤 피처 벡터화를 적용한다.
# 그리고 그 이후에 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가 한다.

### 텍스트 정규화 

In [2]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

In [3]:
# 어떠한 key 값을 가지고 있는지 확인해보자.

print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [4]:
# 타겟 클래스가 어떻게 구성 되어 있는지 확인해보자.

import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
# 개별 데이터가 텍스트로 어떻게 구성돼 있는지 데이터를 한 개만 추출해 값을 확인해보자.

print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [6]:
# 텍스트 데이터를 확인해 보면 뉴스그룹 기사의 내용 뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있다.
# 이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거한다.
# 왜냐하면 제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 떄문이다.
# 이 피처들을 포함하게 되면 왠만한 ML 알고리즘을 적용해도 상당히 높은 예측 성능을 나타낸다.
# 따라서 이들 헤더와 푸터 정보를 포함하는 것은 이 장에서 수행하려는 텍스트 분석의 의도를 벗어나기에 순수한 텍스트만으로 구성된 기사 내용으로
# 어떤 뉴스그룹에 속하는지 분류할 것이다.

In [7]:
# remove 파라미터를 이용하면 뉴스그룹 기사의 헤더(header)와 푸터(footer) 등을 제거할 수 있다.

from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용(Train) 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

# subset='test'으로 테스트(test) 데이터만 추출, remove=('headers, 'footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

<class 'list'>
학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가 

In [8]:
# CountVectorizer를 이용해 학습 데이터의 텍스틀 피처 벡터화 한다. 
# 테스트 데이터 역시 피처 벡터화를 수행하는데, 한가지 유의할 점이 있다.
# 바로 테스트 데이터에서 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된
# CountVectorizer 객체를 이용해 테스트 데이터를 변환(transform) 해야 한다는 것이다.
# 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아진다.

# 테스트 데이터의 피처 벡터화 시 fit_transform()을 사용하면 안된다는 점도 유의한다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행.
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


In [10]:
# 피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스 그룹에 대한 분류를 예측해보자.

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.606


/Users/adam/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


In [12]:
# TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공한다.
# 일반적으로 문서 내에 텍스트가 가장 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출한다.

# 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은,
# 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것이다.

# 텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있다.

In [13]:
# stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용.
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorizaed Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorizaed Logistic Regression의 예측 정확도는 0.692


In [ ]:
# 이번에는 GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행해보자.

from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정. 
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


### 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합 

In [ ]:
# 사이킷런의 Pipeline 클래스를 이용하면 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행할 수 있다.
# 일반적으로 머신러닝에서 Pipeline이란 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을
# 마치 '수도관(pipe)에서 물 흐르듯' 한꺼번에 스트림 기반으로 처리한다는 의미이다.
# 이렇게 Pipeline을 이용하면 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있어
# 더 직관적인 ML 모델 코드를 생성할 수 있다.

# 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 
# 바로 머신러닝 알고리즘의 데이터로 입력할 수 있기 떄문에 수행 시간을 절약 할 수 있다.
# 일반적으로 사이킷런 파이프라인은 텍스트 기반의 피처 벡터화뿐 아니라 모든 데이터 전처리 작업과 Estimator를 결합할 수 있다.
# 예를 들어 스케일링 또는 벡터 정규화, PCA 등의 변환 작업과 분류, 회귀 등의 Estimator를 한 번에 결합하는 것이다.

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect', TfidVectorizer(stop_words='english')),
                    ('lr_clf', LogisticRegression(random_state=156))])

In [ ]:
# 이것은 TfidVectorizer 객체를 tfidf_vect라는 객체 변수 명으로, LogisticRegression 객체를 lr_clf라는
# 객체 변수 명으로 생성한 뒤 이 두개의 객체를 파이프라인으로 연결하는 Pipeline 객체 pipeline을 생성한다는 의미이다.
# TfidVectorizer의 피처 벡터화와 LogisticRegressor의 머신러닝 모델의 학습 및 예측이
# Pipeline의 fit()과 predict()로 통일돼 수행됨을 알 수 있다.

In [ ]:
from sklearn.pipeline import Pipeline

#TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
            ('tfidf_Vect', TfidVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)),
            ('lr_clf', LogisticRegrerssion(C=10))
])

# 별도의 TfidVectorizer객체의 fit_transform( )과 LogisticRegression의 fit(), predict()가 필요 없다.
# pipeline의 fit( )과 predict( ) 만으로 한꺼번에 Feature Vectorization과 ML 학습/예측이 가능.
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

In [ ]:
# 사이킷런은 GridSearchCV 클래스의 생성 파라미터로 Pipeline을 입력해 Pipeline 기반에서도 하이퍼 파라미터 튜닝을
# GridSearchCV 방식으로 진행할 수 있게 지원한다. 이렇게 하면 피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터를
# 모두 한 번에 GridSearchCV를 이용해 최적화 할 수 있다.
# Pipeline + GridSearchCV를 적용할 때 유의할 점은 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모된다는 점이다.

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression(solver='liblinear'))
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼 파라미터 이름과 값을 설정.
params = {'tfidf_vect_ngram_range': [(1,1), (1,2), (1,3)],
          'tfidf_vect_max_df': [100, 300, 700],
          'lr_clf_C': [1, 5, 10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

In [ ]:
# 서포트 벡터머신 (Support Vector Machine)과 나이브 베이즈(Naive Bayes) 알고리즘도 희소 행렬 기반의
# 텍스트 분류에 자주 사용되는 머신러닝 알고리즘이다.